In [4]:
import itertools
from collections import OrderedDict

def recurnonincreasing(length,minval,maxval):
    """
    This is a generator that iterates over nonincreasing integer sequences
    of length 'length', with values bounded between 'minval' and 'maxval'.
    """
    for currval in range(minval,maxval+1,1):
        if length == 1:
            yield [currval]
        else:
            for x in recurnonincreasing(length-1,minval,currval):
                x.insert(0,currval)
                yield x


def testtetra(seq):
    """
    Given a list of numbers in the order (x,y,z, xb,yb,zb), 
    where (x,y,z) are the lengths of 3 edges coming out of a vertex and
    (xb,yb,zb) are the three edges opposite them (in order),
    check whether they are edge lengths of a tetrahedron.
    
    See Remark 5 in section 3 of 
    Wirth and Dreiding, Edge lengths determining tetrahedrons, 
    Elem. Math. 64 (2009) 160-170.
    """
    if len(seq) !=6:
        return False
    else:
        x,y,z,xb,yb,zb = seq
        # 2D Cayley-Menger for face (xb,yb,zb)
        D1S = -(xb+yb+zb)*(xb+yb-zb)*(yb+zb-xb)*(zb+xb-yb)
        if D1S >= 0:
            #print('D1S failed')
            return False
        else:
            x2,y2,z2,xb2,yb2,zb2 = [n*n for n in seq]
            # 3D Cayley-Menger
            DS = (2*x2*xb2*(y2+yb2+z2+zb2-x2-xb2) + 
                2*y2*yb2*(z2+zb2+x2+xb2-y2-yb2) +
                2*z2*zb2*(x2+xb2+y2+yb2-z2-zb2) +
                (x2-xb2)*(x2-xb2)*(z2-zb2)-(x2+xb2)*(y2+yb2)*(z2+zb2))
            if DS <= 0:
                #print('DS failed')
                return False
            else:
                return True

def tetraperm(nonincseq):
    """
    Given a nonincreasing sequence of 6 integers, this generator returns the 
    30 nonisometric orderings suitable for input into testtetra.
    
    This is based on the discussion at the beginning of section 6 of
    Wirth and Dreiding, Edge lengths determining tetrahedrons, 
    Elem. Math. 64 (2009) 160-170).
    """
    longest = nonincseq[0]
    for k in range(5):
        remaining = nonincseq[1:]
        opposite = remaining.pop(k)
        nextlargest = max(remaining)
        remaining.remove(nextlargest)
        for perm in itertools.permutations(remaining):
            yield [longest, nextlargest, perm[0], opposite, perm[1], perm[2]]

In [91]:
# max integer length
m=5

bigtable = [seq for noninc in recurnonincreasing(6,1,m) 
            for seq in tetraperm(noninc) if testtetra(seq)]

tups = [tuple(ent) for ent in bigtable]
#data=list(set(tups))
orderedlengths = list(OrderedDict.fromkeys(tups))

lengthsonly = [noninc for noninc in recurnonincreasing(6,1,m) 
              if all([testtetra(seq) for seq in tetraperm(noninc)])]

In [92]:
len(orderedlengths), len(lengthsonly)

(392, 41)

In [94]:
# list of sets of 6 strictly positive integers
# which are the lengths of the edges of at least one tetrahedron
lengthsonly

[[1, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 2],
 [3, 2, 2, 2, 2, 2],
 [3, 3, 3, 2, 2, 2],
 [3, 3, 3, 3, 2, 2],
 [3, 3, 3, 3, 3, 1],
 [3, 3, 3, 3, 3, 2],
 [3, 3, 3, 3, 3, 3],
 [4, 3, 3, 3, 3, 2],
 [4, 3, 3, 3, 3, 3],
 [4, 4, 3, 3, 3, 3],
 [4, 4, 4, 3, 3, 2],
 [4, 4, 4, 3, 3, 3],
 [4, 4, 4, 4, 3, 2],
 [4, 4, 4, 4, 3, 3],
 [4, 4, 4, 4, 4, 1],
 [4, 4, 4, 4, 4, 2],
 [4, 4, 4, 4, 4, 3],
 [4, 4, 4, 4, 4, 4],
 [5, 3, 3, 3, 3, 3],
 [5, 4, 4, 3, 3, 3],
 [5, 4, 4, 4, 3, 3],
 [5, 4, 4, 4, 4, 2],
 [5, 4, 4, 4, 4, 3],
 [5, 4, 4, 4, 4, 4],
 [5, 5, 4, 4, 4, 3],
 [5, 5, 4, 4, 4, 4],
 [5, 5, 5, 4, 3, 3],
 [5, 5, 5, 4, 4, 2],
 [5, 5, 5, 4, 4, 3],
 [5, 5, 5, 4, 4, 4],
 [5, 5, 5, 5, 3, 3],
 [5, 5, 5, 5, 4, 2],
 [5, 5, 5, 5, 4, 3],
 [5, 5, 5, 5, 4, 4],
 [5, 5, 5, 5, 5, 1],
 [5, 5, 5, 5, 5, 2],
 [5, 5, 5, 5, 5, 3],
 [5, 5, 5, 5, 5, 4],
 [5, 5, 5, 5, 5, 5]]

In [93]:
# list of ordered tuples of 6 strictly positive integers
# which are the lengths of the edges of tetrahedra
# given in the order (x,y,z, xb,yb,zb), 
# where (x,y,z) are the lengths of 3 edges coming out of a vertex and
# (xb,yb,zb) are the three edges opposite them (in order)
orderedlengths

[(1, 1, 1, 1, 1, 1),
 (2, 2, 2, 1, 1, 1),
 (2, 2, 1, 2, 2, 1),
 (2, 2, 2, 2, 2, 1),
 (2, 2, 2, 2, 1, 2),
 (2, 2, 1, 2, 2, 2),
 (2, 2, 2, 1, 2, 2),
 (2, 2, 2, 2, 2, 2),
 (3, 2, 2, 1, 1, 1),
 (3, 2, 2, 2, 2, 1),
 (3, 2, 2, 1, 2, 2),
 (3, 2, 2, 2, 2, 2),
 (3, 3, 2, 2, 2, 1),
 (3, 3, 2, 2, 2, 2),
 (3, 3, 3, 1, 1, 1),
 (3, 3, 3, 2, 2, 1),
 (3, 3, 3, 2, 1, 2),
 (3, 3, 3, 1, 2, 2),
 (3, 3, 2, 3, 2, 2),
 (3, 3, 3, 2, 2, 2),
 (3, 3, 2, 2, 3, 2),
 (3, 3, 2, 2, 2, 3),
 (3, 3, 1, 3, 3, 1),
 (3, 3, 2, 3, 3, 1),
 (3, 3, 1, 3, 3, 2),
 (3, 3, 3, 3, 2, 2),
 (3, 3, 2, 3, 3, 2),
 (3, 3, 2, 3, 2, 3),
 (3, 3, 3, 2, 3, 2),
 (3, 3, 3, 2, 2, 3),
 (3, 3, 2, 2, 3, 3),
 (3, 3, 3, 3, 3, 1),
 (3, 3, 3, 3, 1, 3),
 (3, 3, 1, 3, 3, 3),
 (3, 3, 3, 1, 3, 3),
 (3, 3, 3, 3, 3, 2),
 (3, 3, 3, 3, 2, 3),
 (3, 3, 2, 3, 3, 3),
 (3, 3, 3, 2, 3, 3),
 (3, 3, 3, 3, 3, 3),
 (4, 2, 2, 1, 1, 1),
 (4, 3, 2, 1, 1, 1),
 (4, 3, 3, 1, 1, 1),
 (4, 3, 3, 2, 2, 1),
 (4, 3, 3, 2, 1, 2),
 (4, 3, 3, 1, 2, 2),
 (4, 3, 3, 2, 2, 2),
 (4, 3, 2, 2,